In [1]:
import re
import pandas as pd

In [5]:
LOG_DIR = "../log"
DECOY_CONFIG_PATH = "../decoyConfig.txt"

In [21]:
previewLogFile = open(LOG_DIR+"/preview.log", "r")
texteditLogFile = open(LOG_DIR+"/textedit.log", "r")
decoyConfigFile = open(DECOY_CONFIG_PATH, "r")

previewLogContent = previewLogFile.readlines()
texteditLogContent = texteditLogFile.readlines()
decoyConfigContent = decoyConfigFile.readlines()
decoyList = []
for filename in decoyConfigContent:
    if "\n" in filename:
        decoyList.append(filename[:-1])
    else:
        decoyList.append(filename)
print decoyList

['scannedSSN.pdf', 'scannedSSN1.pdf', 'scannedSSN2.pdf', 'scannedSSN3.pdf', 'scannedSSN4.pdf', 'scannedSSN5.pdf', 'Junyi_Driver_License.jpg', 'Junyi_Driver_License1.jpg', 'Junyi_Driver_License2.jpg', 'Junyi_Driver_License4.jpg', 'Junyi_Driver_License5.jpg', 'Junyi_Wang_Unofficial_Transcript.pdf', 'Nov_BOA_Bank_Statement.pdf', 'Oath_Signed_With_Passport.pdf', 'security-deposit-check-copy.pdf', 'BOA-check-both-side.pdf', 'i20-opt.pdf', 'SocialSecurityNumber.pdf', 'online-account.txt', 'passport-copy.jpeg', 'columbia-id.jpg', 'columbia-id2.jpg', 'UndergradResume2.pdf', 'UndergradResume.pdf']


In [7]:
previewLogContent = [line for line in previewLogContent if "---5min Preview" in line or ".jpg" in line or ".jpeg" in line or ".pdf" in line or ".png" in line]
texteditLogContent = [line for line in texteditLogContent if "---5min TextEdit" in line or ".txt" in line]

In [8]:
def previewSplitByMilestone(inputList):    
    copy = False
    buffer = ""
    splitByMilestone = []
    for line in inputList:
        if "---5min Preview" in line:
            copy = True
            splitByMilestone.append(buffer)
            buffer = ""
        if copy:
            buffer += line.split("/")[-1]
    return splitByMilestone

In [9]:
def texteditSplitByMilestone(inputList):
    copy = False
    buffer = ""
    splitByMilestone = []
    for line in inputList:
        if "---5min TextEdit" in line:
            copy = True
            splitByMilestone.append(buffer)
            buffer = ""
        if copy:
            buffer += line.split("/")[-1]
    return splitByMilestone

In [10]:
previewByMilestone = previewSplitByMilestone(previewLogContent)[1:]
texteditByMilestone = texteditSplitByMilestone(texteditLogContent)[1:]

In [31]:
def calFileTouched(previewByMilestone, texteditByMilestone, decoyList):
    docTouchedList = []
    imgTouchedList = []
    decoyTouchedList = []
    for i in range(len(previewByMilestone)):
        docTouched = 0
        imgTouched = 0
        decoyTouched = 0
        
        previewSet = set(previewByMilestone[i].split("\n")[1:])
        for eachFile in previewSet:
#             print eachFile.strip()
            if ".jpg" in eachFile or ".jpeg" in eachFile or ".png" in eachFile:
                imgTouched += 1
            if ".pdf" in eachFile:
                docTouched += 1
            if eachFile.strip() in decoyList:
                print eachFile
                decoyTouched += 1
        
        texteditSet = set(texteditByMilestone[i].split("\n")[1:])
        for eachFile in texteditSet:
            if ".txt" in eachFile:
                docTouched += 1
            if eachFile.strip() in decoyList:
                print eachFile
                decoyTouched += 1
                
        docTouchedList.append(docTouched)
        imgTouchedList.append(imgTouched)
        decoyTouchedList.append(decoyTouched)
        
    return docTouchedList, imgTouchedList, decoyTouchedList
        
            

In [32]:
docTouchedList, imgTouchedList, decoyTouchedList = calFileTouched(previewByMilestone, texteditByMilestone, decoyList)

security-deposit-check-copy.pdf 
Junyi_Driver_License.jpg 
scannedSSN2.pdf 
Nov_BOA_Bank_Statement.pdf 
Oath_Signed_With_Passport.pdf 


In [33]:
for i in range(len(docTouchedList)):
    print previewByMilestone[i].split("\n")[0]
    print("Document Touched: " + str(docTouchedList[i]))
    print("Image Touched: " + str(imgTouchedList[i]))
    print("Decoy Touched: " + str(decoyTouchedList[i]))
    print("\n")

---5min Preview Audit milestone--- 2018-12-03-16:45:10
Document Touched: 0
Image Touched: 0
Decoy Touched: 0


---5min Preview Audit milestone--- 2018-12-03-16:50:10
Document Touched: 1
Image Touched: 0
Decoy Touched: 1


---5min Preview Audit milestone--- 2018-12-03-16:55:10
Document Touched: 1
Image Touched: 2
Decoy Touched: 2


---5min Preview Audit milestone--- 2018-12-03-17:00:11
Document Touched: 1
Image Touched: 0
Decoy Touched: 0


---5min Preview Audit milestone--- 2018-12-03-17:05:11
Document Touched: 1
Image Touched: 1
Decoy Touched: 0


---5min Preview Audit milestone--- 2018-12-03-17:10:12
Document Touched: 2
Image Touched: 0
Decoy Touched: 0


---5min Preview Audit milestone--- 2018-12-03-17:15:12
Document Touched: 5
Image Touched: 0
Decoy Touched: 0


---5min Preview Audit milestone--- 2018-12-03-17:20:12
Document Touched: 1
Image Touched: 1
Decoy Touched: 0


---5min Preview Audit milestone--- 2018-12-03-17:25:13
Document Touched: 0
Image Touched: 0
Decoy Touched: 0


-

In [34]:
df1 = pd.DataFrame(docTouchedList, columns=['Num of doc touched'])
df2 = pd.DataFrame(imgTouchedList, columns=['Num of img touched'])
df3 = pd.DataFrame(decoyTouchedList, columns=['Num of decoy touched'])
df1.to_csv("doc_file_monitor.csv", index=False, sep=' ')
df2.to_csv("img_file_monitor.csv", index=False, sep=' ')
df3.to_csv("decoy_file_monitor.csv", index=False, sep=' ')